In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")  # or any other suitable solver, like CPLEX, Gurobi, etc.

   Resolving package versions...
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`


In [2]:
Pkg.add("Distributions")
using Distributions

   Resolving package versions...
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`


In [25]:
#function to calculate replacement cost
function ReplacementCost(initialcost, mincost, years, age, depreciation_rate)
    dep_years = years[age:end]
    for i = 1:length(dep_years)
        costs[i] = initialcost * depreciation_rate^(-1 * (dep_years[i])) + mincost
    end
    return costs
 end
 #Test example for how to use function
 #=years = collect(1:15)
 age = 3
 dep_years = years[age:end]
 initialcost = 25000
 mincost = 12500
 depreciation_rate = 1.5
 costs = zeros(1,length(dep_years))
 for i = 1:length(dep_years)
    costs[i] = initialcost * depreciation_rate^(-1 * (dep_years[i])) + mincost
 end=#

ReplacementCost (generic function with 1 method)

In [38]:
#create fleet (will replace code in next block)
function newFleet(sedans, hybrids, mini, vans, trucks) #inputs are number of each vehicle
    s = zeros(sedans, 13)
    for i in 1:sedans
        a = rand(Uniform(1,14)) #create a random age (this is technically the years left until it should be replaced but it works the same)
        for t in 1:13 #for all years
            if t < 8 #if gas cars are still allowed
                if t < a #if the car is younger than it's replacement age
                    s[i,t] = 10000 * 2^(-.5*t) + 20000 
                    #current year average cost of EV sedan with slow depreciation to expected future costs
                else
                    s[i,t] = 10000 * 2^(-.5*t) + 50000 #$30,000 penalty
                end
            else
                s[i,t] = 1000000000 #lower if we want an economic punishment instead of an absolute rule
            end
        end
    end #repeat for all other types of cars
    h = zeros(hybrids, 13)
    for i in 1:hybrids
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < 8
                if t < a
                    h[i,t] = 15000 * 1.9^(-.5*t) + 23000
                    #values change depending on type of vehicle, i.e. hybrids cost more
                    #and will reduce in cost slower than sedans
                else
                    h[i,t] = 15000 * 1.9^(-.5*t) + 53000
                end
            else
                h[i,t] = 1000000000 #lower if we want an economic punishment instead of an absolute rule
            end
        end
    end
    m = zeros(mini, 13)
    for i in 1:mini
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < a
                m[i,t] = 10000 * 1.6^(-.5*t) + 30000
            else
                m[i,t] = 10000 * 1.6^(-.5*t) + 60000
            end      
        end
    end
    v = zeros(vans, 13)
    for i in 1:vans
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < a
                v[i,t] = 10000 * 1.6^(-.5*t) + 35000
            else
                v[i,t] = 10000 * 1.6^(-.5*t) + 65000
            end      
        end
    end
    tr = zeros(trucks, 13)
    for i in 1:trucks
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < a
                tr[i,t] = 30000 * 1.5^(-.5*t) + 50000
            else
                tr[i,t] = 30000 * 1.5^(-.5*t) + 80000
            end      
        end
    end
    fleet = [s,h,m,v,tr]
    return fleet
end

newFleet (generic function with 1 method)

In [41]:
#generate all variables
fleet = newFleet(3, 3, 3, 3, 4)
fleetMiles = [4000,4000,4000,4000,12000] #based on US survey, car and van miles are 1/3 because fleet services says they are only checked out roughly 1/3 the time while trucks are mostly permanently assigned, same order as variables are initialized
gasCost = [0.18, 12, 23, 32, 24] #rough dollars of gas per mile driven, same order as above
elecCost = [0,0,0,0,0] #rough dollars of electricity per mile driven, same order
newGas = [0,0,0,0,0] #cost of buying a new gas vehicle of type n, same order
newElec = [0,0,0,0,0] #cost of buying a new electric vehicle of type n, same order

5-element Vector{Int64}:
 0
 0
 0
 0
 0

In [42]:
function travelCost(fleet, t)
    totalYearlyCost = 0
    for i in 1:length(fleet)
        for k in 1:size(fleet[i],2)
            for t in 1:13
                if fleet[i][2,k,t] == 0 #check for electric
                    totalYearlyCost += fleetMiles[i]*gasCost[i]
                else 
                    totalYearlyCost += fleetMiles[i]*elecCost[i] #could look to add a year modifyer to decrease electricity costs
                end
            end
        end
    end
end


travelCost (generic function with 1 method)

In [3]:
#possible replacement for travelCost function above
function fuelCost(type, electric) #type is a number corresponding to the array index, electric is a boolean
    fleetMiles = [4000,4000,4000,4000,12000] #based on US survey, car and van miles are 1/3 because fleet services says they are only checked out roughly 1/3 the time while trucks are mostly permanently assigned, same order as variables are initialized
    gasCost = [0.18, 0.12, 0.23, 0.32, 0.24] #rough dollars of gas per mile driven, same order as above
    elecCost = [0,0,0,0,0] #rough dollars of electricity per mile driven, same order
    if electric
        return fleetMiles[type]*elecCost[type]
    end
    return return fleetMiles[type]*gasCost[type]
end

fuelCost (generic function with 1 method)

In [7]:
using JuMP
using HiGHS  # or your preferred solver

model = Model(HiGHS.Optimizer)

# Decision variables
@variable(model, )

# Objective function (minimize the total cost)
@objective(model, Min, )

# Constraints (e.g., only one replacement per year)
@constraint(model, electric_goal1, sum(fleet[i] for i in type))

LoadError: LoadError: At In[7]:10: `@objective(model, Min)`: needs three arguments: model, objective sense (Max or Min) and expression.
in expression starting at In[7]:10